In [21]:
import pandas as pd
import numpy as np
import math
from pathlib import Path

from db_psql_model import DatabaseCursor

pd.set_option("display.max_rows", 9999)
pd.set_option("display.max_colwidth", 40)
pd.set_option("display.max_columns", 999)
pd.set_option("display.precision", 2)


PATH = list(Path().cwd().parent.glob("**/private.yaml"))[0]
OPTION_DEV = "-c search_path=dev"
OPTION_PROD = "-c search_path=prod"
GAME_ID = 406  #4 Team 2021
GAME_ID = 359  # 6 Team 2016
GAME_ID = 273  # 8 team 2012


league_settings_query = f"SELECT * from dev.league_settings where game_id = '{GAME_ID}'"
league_settings = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(
    league_settings_query
)

num_playoff_teams = league_settings["num_playoff_teams"].values[0]

playoff_start_week = league_settings["playoff_start_week"].values[0]

matchup_board_query = f"SELECT * FROM prod.matchup_board_{int(GAME_ID)}"
matchup_board = DatabaseCursor(PATH, options=OPTION_PROD).copy_data_from_postgres(
    matchup_board_query
)

team_points_weekly_query = f"SELECT * from dev.weekly_team_pts where game_id = '{GAME_ID}' and week >= {playoff_start_week}"
team_points_weekly = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(
    team_points_weekly_query
).drop_duplicates()

playoff_teams = list(
    matchup_board["team_key"][matchup_board["reg_season_rank"] <= num_playoff_teams]
)

team_points_weekly["team_key"] = (
    team_points_weekly["game_id"].astype(str)
    + ".l."
    + team_points_weekly["league_id"].astype(str)
    + ".t."
    + team_points_weekly["team_id"].astype(str)
)

team_points_weekly = team_points_weekly[
    team_points_weekly["team_key"].isin(playoff_teams)
]
team_points_weekly.sort_values(["week", "team_id"])

Successfully pulled: SELECT * from dev.league_settings where game_id = '273'
Successfully pulled: SELECT * FROM prod.matchup_board_273
Successfully pulled: SELECT * from dev.weekly_team_pts where game_id = '273' and week >= 15


,final_points,game_id,league_id,projected_points,team_id,week,team_key
0,65.86,273,777818,102.73,1,15,273.l.777818.t.1
1,71.90,273,777818,58.54,2,15,273.l.777818.t.2
3,75.38,273,777818,85.51,4,15,273.l.777818.t.4
5,77.14,273,777818,90.89,6,15,273.l.777818.t.6
6,88.50,273,777818,106.62,7,15,273.l.777818.t.7
8,63.68,273,777818,110.55,9,15,273.l.777818.t.9
9,106.34,273,777818,117.06,10,15,273.l.777818.t.10
11,131.12,273,777818,96.76,12,15,273.l.777818.t.12
24,135.44,273,777818,96.65,1,16,273.l.777818.t.1
25,69.02,273,777818,71.45,2,16,273.l.777818.t.2


In [2]:
def printMatches(matches):
    print("Active Matches:")
    for match in matches:
        if match.is_ready_to_start():
            print(
                "\t{} vs {}".format(
                    *[p.get_competitor() for p in match.get_participants()]
                )
            )


def add_win(tourney, competitor):
    m = tourney.get_active_matches_for_competitor(competitor)[0]
    tourney.add_win(m, competitor)


def checkActiveMatches(tourney, competitorPairs):
    matches = tourney.get_active_matches()
    if len(competitorPairs) != len(matches):
        printMatches(matches)
        print(competitorPairs)
        raise Exception(
            "Invalid number of competitors: {} vs {}".format(
                len(matches), len(competitorPairs)
            )
        )
    for match in matches:
        inMatches = False
        for competitorPair in competitorPairs:
            participants = match.get_participants()
            if competitorPair[0] == participants[0].get_competitor():
                if competitorPair[1] == participants[1].get_competitor():
                    inMatches = True
            elif competitorPair[0] == participants[1].get_competitor():
                if competitorPair[1] == participants[0].get_competitor():
                    inMatches = True
    if not inMatches:
        printMatches(matches)
        print(competitorPairs)
        # raise Exception("Wrong matches")

In [3]:
class Participant:
    """
    The Participant class represents a participant in a specific match.
    It can be used as a placeholder until the participant is decided.
    """

    def __init__(self, competitor=None):
        self.competitor = competitor

    def get_competitor(self):
        """
        Return the competitor that was set,
        or None if it hasn't been decided yet
        """
        return self.competitor

    def set_competitor(self, competitor):
        """
        Set competitor after you've decided who it will be,
        after a previous match is completed.
        """
        self.competitor = competitor


class Match:
    """
    A match represents a single match in a tournament, between 2 participants.
    It adds empty participants as placeholders for the winner and loser,
    so they can be accessed as individual object pointers.
    """

    def __init__(self, left_participant, right_participant):
        self.__left_participant = left_participant
        self.__right_participant = right_participant
        self.__winner = Participant()
        self.__loser = Participant()

    def set_winner(self, competitor):
        """
        When the match is over, set the winner competitor here and the loser will be set too.
        """
        if competitor == self.__left_participant.get_competitor():
            self.__winner.set_competitor(competitor)
            self.__loser.set_competitor(self.__right_participant.get_competitor())
        elif competitor == self.__right_participant.get_competitor():
            self.__winner.set_competitor(competitor)
            self.__loser.set_competitor(self.__left_participant.get_competitor())
        else:
            raise Exception("Invalid competitor")

    def get_winner_participant(self):
        """
        If the winner is set, get it here. Otherwise this return None.
        """
        return self.__winner

    def get_loser_participant(self):
        """
        If the winner is set, you can get the loser here. Otherwise this return None.
        """
        return self.__loser

    def get_participants(self):
        """
        Get the left and right participants in a list.
        """
        return [self.__left_participant, self.__right_participant]

    def is_ready_to_start(self):
        """
        This returns True if both of the participants coming in have their competitors "resolved".
        This means that the match that the participant is coming from is finished.
        It also ensure that the winner hasn't been set yet.
        """
        is_left_resolved = self.__left_participant.get_competitor() is not None
        is_right_resolved = self.__right_participant.get_competitor() is not None
        is_winner_resolved = self.__winner.get_competitor() is not None
        return is_left_resolved and is_right_resolved and not is_winner_resolved


class Tournament:
    """
    This is a single-elimination tournament where each match is between 2 competitors.
    It takes in a list of competitors, which can be strings or any type of Python object,
    but they should be unique. They should be ordered by a seed, with the first entry being the most
    skilled and the last being the least. They can also be randomized before creating the instance.
    Optional options dict fields:
    """

    def __init__(self, competitors_list, options={}):
        assert len(competitors_list) > 1
        self.__matches = []
        next_higher_power_of_two = int(
            math.pow(2, math.ceil(math.log2(len(competitors_list))))
        )
        winners_number_of_byes = next_higher_power_of_two - len(competitors_list)
        incoming_participants = list(map(Participant, competitors_list))
        incoming_participants.extend([None] * winners_number_of_byes)

        while len(incoming_participants) > 1:
            half_length = int(len(incoming_participants) / 2)
            first = incoming_participants[0:half_length]
            last = incoming_participants[half_length:]
            last.reverse()
            next_round_participants = []
            for participant_pair in zip(first, last):
                if participant_pair[1] is None:
                    next_round_participants.append(participant_pair[0])
                elif participant_pair[0] is None:
                    next_round_participants.append(participant_pair[1])
                else:
                    match = Match(participant_pair[0], participant_pair[1])
                    next_round_participants.append(match.get_winner_participant())
                    self.__matches.append(match)
            incoming_participants = next_round_participants
        self.__winner = incoming_participants[0]

    def __iter__(self):
        return iter(self.__matches)

    def get_active_matches(self):
        """
        Returns a list of all matches that are ready to be played.
        """
        return [match for match in self.get_matches() if match.is_ready_to_start()]

    def get_matches(self):
        """
        Returns a list of all matches for the tournament.
        """
        return self.__matches

    def get_active_matches_for_competitor(self, competitor):
        """
        Given the string or object of the competitor that was supplied
        when creating the tournament instance,
        returns a list of Matches that they are currently playing in.
        """
        matches = []
        for match in self.get_active_matches():
            competitors = [
                participant.get_competitor() for participant in match.get_participants()
            ]
            if competitor in competitors:
                matches.append(match)
        return matches

    def get_winners(self):
        """
        Returns None if the winner has not been decided yet,
        and returns a list containing the single victor otherwise.
        """
        if len(self.get_active_matches()) > 0:
            return None
        return [self.__winner.get_competitor()]

    def add_win(self, match, competitor):
        """
        Set the victor of a match, given the competitor string/object and match.
        """
        match.set_winner(competitor)

In [18]:
# round_1 = [[1, 8], [2, 7], [3, 6], [4, 5]]
# round_2 = [[18, 45], [27, 36]]
# round_3 = [[1845, 2736]]
# champion = []
# tourney = Tournament(playoff_teams)
# checkActiveMatches(tourney, round_1)
# add_win(tourney, '273.l.777818.t.7')
# add_win(tourney, '273.l.777818.t.12')
# add_win(tourney, '273.l.777818.t.4')
# add_win(tourney, '273.l.777818.t.6')
# checkActiveMatches(tourney, round_2)
# add_win(tourney, '273.l.777818.t.6')
# add_win(tourney, '273.l.777818.t.4')
# checkActiveMatches(tourney, round_3)

In [23]:
next_higher_power_of_two = int(math.pow(2, math.ceil(math.log2(len(playoff_teams)))))

winners_number_of_byes = next_higher_power_of_two - len(playoff_teams)

incoming_participants = list(playoff_teams)

incoming_participants.extend(["Bye"] * winners_number_of_byes)

num_of_rounds = int(math.ceil(math.log2(len(incoming_participants))))

num_of_matches = int((len(incoming_participants) / 2) * num_of_rounds)

matches_per_round = int(num_of_matches / num_of_rounds)

half_length = int(len(incoming_participants) / 2)

first = incoming_participants[0:half_length]

last = incoming_participants[half_length:]
last.reverse()

round_1_matchups = list(zip(first, last))

round_1 = team_points_weekly[["week", "team_key", "final_points", "projected_points"]][
    (team_points_weekly["week"] == playoff_start_week)
]

round_1 = round_1.merge(
    matchup_board[['game_id', 'reg_season_rank', 'mgr', 'team', 'team_key', 'pts_rank']], how="outer", left_on="team_key", right_on="team_key"
)

round_1 = round_1[
    ["week", "reg_season_rank", "team_key", "final_points", "projected_points"]
][round_1["team_key"].isin(playoff_teams)]

round_1.sort_values("reg_season_rank", inplace=True)

round_1.drop("reg_season_rank", axis=1, inplace=True)

m = 0
for match in range(1, matches_per_round + 1):
    round_1.loc[
        round_1["team_key"] == round_1_matchups[m][0], "r1_matchup_num"
    ] = f"{playoff_start_week}.{match}"

    round_1.loc[
        round_1["team_key"] == round_1_matchups[m][1], "r1_matchup_num"
    ] = f"{playoff_start_week}.{match}"

    m += 1

round_1 = round_1.merge(
    round_1[["team_key", "r1_matchup_num", "final_points", "projected_points"]],
    how="outer",
    left_on="r1_matchup_num",
    right_on="r1_matchup_num",
    suffixes=("", "_opp"),
)

round_1 = round_1[~round_1["r1_matchup_num"].isna()]

round_1["count"] = round_1.groupby("r1_matchup_num")["team_key"].transform("count")

round_1 = round_1[
    (round_1["count"] == 1) | (round_1["team_key"] != round_1["team_key_opp"])
]

round_1.loc[round_1["team_key_opp"] == round_1["team_key"], "final_points_opp"] = 0
round_1.loc[round_1["team_key_opp"] == round_1["team_key"], "projected_points_opp"] = 0
round_1.loc[round_1["team_key_opp"] == round_1["team_key"], "team_key_opp"] = "Bye"

for row in round_1:
    round_1["win_loss_r1"] = np.where(
        round_1["final_points"] > round_1["final_points_opp"],
        "W",
        "L",
    )

round_1.rename(
    columns={
        "week": "week_r1",
        "final_points": "pts_r1",
        "projected_points": "pro_pts_r1",
        "r1_matchup_num": "matchup_r1",
        "team_key_opp": "opp_team_key_r1",
        "final_points_opp": "opp_pts_r1",
        "projected_points_opp": "opp_pro_pts_r1",
    },
    inplace=True,
)

round_1.drop("count", axis=1, inplace=True)

matchup_board = matchup_board.merge(
    round_1,
    how="outer",
    left_on="team_key",
    right_on="team_key",
)

In [24]:
matchup_board.sort_values(["win_loss_r1", "matchup_r1"])

,game_id,reg_season_rank,team_key,team,mgr,2pt_rank,2pt_total,w_l_rank,wins,losses,pts_rank,pts_ttl,pro_pts_rank,pro_pts_ttl,week_1,pts_1,pro_pts_1,opp_key_1,opp_team_1,opp_mgr_1,opp_pts_1,opp_pro_pts_1,win_loss_1,2pt_score_1,week_2,pts_2,pro_pts_2,opp_key_2,opp_team_2,opp_mgr_2,opp_pts_2,opp_pro_pts_2,win_loss_2,2pt_score_2,week_3,pts_3,pro_pts_3,opp_key_3,opp_team_3,opp_mgr_3,opp_pts_3,opp_pro_pts_3,win_loss_3,2pt_score_3,week_4,pts_4,pro_pts_4,opp_key_4,opp_team_4,opp_mgr_4,opp_pts_4,opp_pro_pts_4,win_loss_4,2pt_score_4,week_5,pts_5,pro_pts_5,opp_key_5,opp_team_5,opp_mgr_5,opp_pts_5,opp_pro_pts_5,win_loss_5,2pt_score_5,week_6,pts_6,pro_pts_6,opp_key_6,opp_team_6,opp_mgr_6,opp_pts_6,opp_pro_pts_6,win_loss_6,2pt_score_6,week_7,pts_7,pro_pts_7,opp_key_7,opp_team_7,opp_mgr_7,opp_pts_7,opp_pro_pts_7,win_loss_7,2pt_score_7,week_8,pts_8,pro_pts_8,opp_key_8,opp_team_8,opp_mgr_8,opp_pts_8,opp_pro_pts_8,win_loss_8,2pt_score_8,week_9,pts_9,pro_pts_9,opp_key_9,opp_team_9,opp_mgr_9,opp_pts_9,opp_pro_pts_9,win_loss_9,2pt_score_9,week_10,pts_10,pro_pts_10,opp_key_10,opp_team_10,opp_mgr_10,opp_pts_10,opp_pro_pts_10,win_loss_10,2pt_score_10,week_11,pts_11,pro_pts_11,opp_key_11,opp_team_11,opp_mgr_11,opp_pts_11,opp_pro_pts_11,win_loss_11,2pt_score_11,week_12,pts_12,pro_pts_12,opp_key_12,opp_team_12,opp_mgr_12,opp_pts_12,opp_pro_pts_12,win_loss_12,2pt_score_12,week_13,pts_13,pro_pts_13,opp_key_13,opp_team_13,opp_mgr_13,opp_pts_13,opp_pro_pts_13,win_loss_13,2pt_score_13,week_14,pts_14,pro_pts_14,opp_key_14,opp_team_14,opp_mgr_14,opp_pts_14,opp_pro_pts_14,win_loss_14,2pt_score_14,week_15,pts_15,pro_pts_15,opp_key_15,opp_team_15,opp_mgr_15,opp_pts_15,opp_pro_pts_15,win_loss_15,2pt_score_15,week_16,pts_16,pro_pts_16,opp_key_16,opp_team_16,opp_mgr_16,opp_pts_16,opp_pro_pts_16,win_loss_16,2pt_score_16,week_17,pts_17,pro_pts_17,opp_key_17,opp_team_17,opp_mgr_17,opp_pts_17,opp_pro_pts_17,win_loss_17,2pt_score_17,week_r1,pts_r1,pro_pts_r1,matchup_r1,opp_team_key_r1,opp_pts_r1,opp_pro_pts_r1,win_loss_r1
0,273,1,273.l.777818.t.7,Goons,Jake,1,19,1,11,3,2,1619.92,3,1553.41,1,105.06,83.85,273.l.777818.t.12,273.l.777818.t.12,273.l.777818.t.12,82.64,93.65,W,1,2,96.46,96.64,273.l.777818.t.8,273.l.777818.t.8,273.l.777818.t.8,90.50,96.70,W,0,3,78.80,98.21,273.l.777818.t.9,273.l.777818.t.9,273.l.777818.t.9,71.92,112.04,W,0,4,114.56,88.40,273.l.777818.t.10,273.l.777818.t.10,273.l.777818.t.10,101.04,104.59,W,1,5,93.60,100.68,273.l.777818.t.11,La Flama Blanca,Ryan,60.76,102.69,W,0,6,101.16,91.94,273.l.777818.t.1,Taco,James,111.28,74.56,L,0,7,114.88,93.80,273.l.777818.t.2,273.l.777818.t.2,273.l.777818.t.2,88.92,102.46,W,1,8,94.78,97.84,273.l.777818.t.3,Cudde2,Tim,83.50,83.63,W,1,9,94.58,92.55,273.l.777818.t.4,The Fear Boners,Wes,97.66,85.75,L,0,10,111.04,111.20,273.l.777818.t.5,Role players for day,Pat,105.84,85.23,W,1,11,104.64,90.66,273.l.777818.t.6,ELE,Pete,78.82,72.39,W,1,12,79.82,98.40,273.l.777818.t.12,273.l.777818.t.12,273.l.777818.t.12,80.92,86.40,L,0,13,102.30,107.34,273.l.777818.t.8,273.l.777818.t.8,273.l.777818.t.8,78.80,90.23,W,1,14,128.08,98.93,273.l.777818.t.9,273.l.777818.t.9,273.l.777818.t.9,75.84,99.01,W,1,15.0,88.50,106.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,111.66,96.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,88.50,106.62,15.1,273.l.777818.t.12,131.12,96.76,L
6,273,7,273.l.777818.t.1,Taco,James,8,11,7,6,8,5,1453.28,10,1345.51,1,99.28,92.95,273.l.777818.t.2,273.l.777818.t.2,273.l.777818.t.2,128.70,85.76,L,0,2,65.64,86.38,273.l.777818.t.3,Cudde2,Tim,73.24,101.41,L,0,3,67.92,87.05,273.l.777818.t.4,The Fear Boners,Wes,97.80,94.70,L,0,4,117.76,79.60,273.l.777818.t.5,Role players for day,Pat,74.86,95.68,W,1,5,106.22,71.65,273.l.777818.t.6,ELE,Pete,89.44,102.89,W,1,6,111.28,74.56,273.l.777818.t.7,Goons,Jake,101.16,91.94,W,1,7,37.42,75.55,273.l.777818.t.8,273.l.777818.t.8,273.l.777818.t.8,78.18,88.32,L,0,8,80.28,89.59,273.l.777818.t.9,273.l.777818.t.9,273.l.777818.t.9,62.86,97.54,W,0,9,109.04,78.75,273.l.777818.t.10,273.l.777